# Hyperparameter Optimisation Lab - 3

The [HyperOpt](https://github.com/hyperopt/hyperopt) package, developed with support from leading government, academic and private institutions, offers a promising and easy-to-use implementation of a Bayesian hyperparameter optimization algorithm. In this notebook, we will cover how to perform hyperparameter optimization using a sequential model-based optimization (SMBO) technique implemented in the HyperOpt Python package.

Sequential model-based optimization is a Bayesian optimization technique that uses information from past trials to inform the next set of hyperparameters to explore, and there are two variants of this algorithm used in practice:one based on the Gaussian process and the other on the Tree Parzen Estimator.

We start by importing all packages needed by the notebook.

In [1]:
import torch
import time
import datetime

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from mnist_cnn import load_mnist
from mnist_cnn import Net
from mnist_cnn import train
from mnist_cnn import test
from datetime import timedelta

from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin

# This degrades the performance but enforces reproducibility.
torch.use_deterministic_algorithms(True)

# Note: This notebook hasn't been fully tested with GPU backends.
# If you are using GPU-acceleration you may also have to enable the following lines:
#
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False
#torch.cuda.manual_seed_all(1234)

# Make sure our tests are reproducible
torch.manual_seed(1234)
np.random.seed(1234)

if torch.cuda.is_available():
  os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
  print("CUDA available. Using GPU acceleration.\n")
  device = "cuda"
else:
  print("CUDA is NOT available. Using CPU for training.\n")
  device = "cpu"

CUDA is NOT available. Using CPU for training.



Loading the training, validation and test sets. Note, that we are only loading the first 20,000 samples from MNIST. This is not a big problem as the samples are relatively balanced across the 10 possible classes. If you'd like to stay on the safe side, you can use the following code to double check the distribution of classes:

```
import matplotlib.pyplot as plt
plt.hist(train_y.numpy(), bins=10)
plt.show()
```

In [2]:
train_X, train_y, valid_X, valid_y, test_X, test_y = load_mnist(device)

train_X = train_X[:20000]
train_y = train_y[:20000]

train_X size :  torch.Size([48000, 1, 28, 28])
train_y size :  torch.Size([48000])
valid_X size :  torch.Size([12000, 1, 28, 28])
valid_y size :  torch.Size([12000])
test_X size  :  torch.Size([10000, 1, 28, 28])
test_y size  :  torch.Size([10000])


Next, we create the CNN network that we'll be fitting on MNIST.

In [3]:
net = Net(activ = F.relu)
net.to(device)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

HyperOpt requires the following key information to execute a hyperparameter search:
* **objective function** - a function that the search will try to minimise
* **search space** - they hyperparameter space to search over
* **data store** - location to persist the search results
* **search algorithm** - a search algorithm for selection of hyperparameter combinations


Let's start by defining the **search space**. A search space consists of nested function expressions, including stochastic expressions. The stochastic expressions are the hyperparameters. The hyperparameter optimization algorithms work by replacing normal "sampling" logic with adaptive exploration strategies. Stochastic expressions normally have the form of 

```
from hyperopt import hp

hp.<sampling_type>(<label>, <range>)
```

Some commonly used stochastic expressions are:

* hp.choice(label, options) --- Returns one of the options, which should be a list or tuple.

* hp.randint(label, upper) --- Returns a random integer in the range [0, upper). The semantics of this distribution is that there is no more correlation in the loss function between nearby integer values, as compared with more distant integer values. This is an appropriate distribution for describing random seeds for example. 

* hp.uniform(label, low, high) --- Returns a value uniformly between low and high. When optimizing, this variable is constrained to a two-sided interval.

* hp.normal(label, mu, sigma) --- Returns a real value that's normally-distributed with mean mu and standard deviation sigma. When optimizing, this is an unconstrained variable.

For example, a bounded range from -10 to +10 for a hyperparameter named "x" can be describe with a search space:

```
space = hp.uniform('x', -10, 10)
```

### Task 1

Define the following space:

| Hyperparameter name | Sampling strategy                |
| ------------------- | -------------------------------- |
| learning rate       | uniform [0.0001, 0.1]            |
| batch size          | one of [10, 20, 50, 100]         |
| momentum            | constant [0.9]                   |
| optimizer           | constant [optim.SGD]             |
| loss_crierion       | constant [nn.CrossEntropyLoss()] |


All hyperparameters should be packaged in a dictionary object like this:

```
search_space = { "<hyperparameter1_name>" : hp.<sampling_type>("<hyperparameter1_name>", <range>),
                 "<hyperparameter2_name>" : hp.<sampling_type>("<hyperparameter2_name>", <range>),
                 ...
               }
```

In [4]:
search_space = {"learning_rate" : hp.uniform("learning_rate", 0.0001, 0.1),
                "batch_size"    : hp.choice("batch_size", [10, 20, 50, 100]),
                "momentum"      : 0.9,
                "optimizer"     : optim.SGD,
                "loss_crierion" : nn.CrossEntropyLoss()
                }

### Task 2

Each iteration in HyperOpt includes a call to the **objective function**. The function receives a point (a combination of parameters) from the search space and returns a loss (negative utility), which HyperOpt tries to minimise.

Your next task is to define the objective function for optimising the hyperparameters of our CNN. While working on it, keep the following in mind:

* All the hyperparameters (including the optimiser and the loss_criterion) are provided to the function via the *params* dictionary
* The function should return the following dictionary as a result of its execution:

```
{ loss : <loss_on_the_validation_set>,
  params : <parameters_used_for_training_the_CNN>,
  status : STATUS_OK
}
```
* Use the *train(\<net\>, \<batch_size\>, \<loss_criterion\>, \<optimizer\>, \<train_X\>, \<train_y\>, verbose = False)* to train the network. Here we set *verbose* to False to reduce the amount of information the network outputs during training.

* To measure the accuracy on the validation set you can use the function 

```
accuracy = test(<net>, valid_X, valid_y)
```

* You may want to print the validation accuracy after each function call so that you have a sense of how the optimisation is progressing

* Don't forget to reset the network with each function call or your results won't be reproducible (use *net.reset()*)

**NOTE**: 
1. Remember that HyperOpt will try to **minimise** the objective. In this case we are using it on the validation accuracy, which we want to maximise instead. You'll have to correct for this.
2. Remember that some of the parameters passed to the **objective function** are optimiser parameters (you'll have to construct a proper optimiser object)

In [5]:
def objective(params):
    
    net.reset()
    
    batch_size = params["batch_size"]
    loss_criterion = params["loss_crierion"]
    
    optimizer = params["optimizer"](net.parameters(), lr=params["learning_rate"], momentum=params["momentum"])
    
    loss_hist = train(net, batch_size, loss_criterion, optimizer, train_X, train_y, verbose = False)
    
    
    val_acc = test(net, valid_X, valid_y)
    
    print("Run completed. Validation accuracy: {:.4f}".format(val_acc))
    print(20*"-")
    
    return {"loss" : -val_acc, "params" : params, "status" : STATUS_OK} 

The final two elements that we need to define are the **search algorithm** and the **data store**.

HyperOpt currently supports three search algorithms:

* Random Search (hyperopt.random.suggest) --- [relevant paper](http://www.jmlr.org/papers/v13/bergstra12a.html)
* Tree of Parzen Estimators (TPE) (hyperopt.tpe.suggest) --- [relevant paper](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)
* Adaptive TPE (ATPE) (hyperopt.atpe.suggest) --- [relevant paper](https://journals.sagepub.com/doi/pdf/10.1177/0020294020932347)

In this tutorial we will use *hyperopt.tpe.suggest* so let's set it as our search algorithm.

In [6]:
algorithm = tpe.suggest

HyperOpt provides two storage mechanisms:

* Trials --- a pure-Python implementation a the storage database using lists of dictionaries.
* MongoTrials --- implements the same API in terms of a mongodb database running in another process.

The elements of *Trials* represent all of the completed, in-progress, and scheduled evaluation points from an e.g. `fmin` call.

Each element a dictionary with *at least* the following keys:

* **tid**: a unique trial identification object within this Trials instance usually it is an integer, but it isn't obvious that other sortable, hashable objects couldn't be used at some point.
      
* **result**: a sub-dictionary representing what was returned by the fmin evaluation function. This sub-dictionary has a key 'status' with a value from `STATUS_STRINGS` and the status is `STATUS_OK`, then there should be a 'loss' key as well with a floating-point value.  

* **misc**: despite generic name, this is currently where the trial's hyperparameter assignments are stored.

In this tutorial we'll use the *Trials* database.

In [7]:
bayes_trials = Trials()

### Task 3

Now that we have all prerequisites in place, we can finally kick off the optimisation process via a call to the *fmin* function. *fmin* explores a function over a hyperparameter space according to a given algorithm. The key arguments that we need to provide are:

* **objecitve** --- This function is called with a value generated from **search** as the first and possibly only argument.  It can return either a scalar-valued loss, or a dictionary.  A returned dictionary must contain a 'status' key with a value from `STATUS_STRINGS`, must contain a 'loss' key if the status is `STATUS_OK`. In our case the **objecitve** is simply called *objective* (This is the function we defined in Task 2).

* **space** --- The set of possible arguments to **objective**. In our case this is *search_space* (Task 1)

* **algo** --- This object, such as `hyperopt.rand.suggest` and `hyperopt.tpe.suggest` provides logic for sequential search of the hyperparameter space. We will use the *algorithm* variable here.

* **max_evals** --- Allow up to this many function evaluations before returning. We will use the value 5 here.

* **trials** --- Storage for completed, ongoing, and scheduled evaluation points. We will use *bayes_trials*.

* **rstate** --- Each call to **algo** requires a seed value, which should be different on each call. Here we'll clamp it to ```np.random.default_rng(1234)``` to ensure reproducibility.

OK, let's now run *fmin* and store its output in a variable called *best_params*. Use the code cell below to make the function call.

In [8]:
best_params = fmin(objective, search_space, algo=algorithm, max_evals=20, trials=bayes_trials, rstate=np.random.default_rng(1234))

Run completed. Validation accuracy: 0.0962            
--------------------                                  
Run completed. Validation accuracy: 0.0990                                        
--------------------                                                              
Run completed. Validation accuracy: 0.0990                                        
--------------------                                                             
Run completed. Validation accuracy: 0.0990                                       
--------------------                                                             
Run completed. Validation accuracy: 0.0962                                       
--------------------                                                             
Run completed. Validation accuracy: 0.0962                                       
--------------------                                                             
Run completed. Validation accuracy: 0.0990                         

Now that the search is complete, we can inspect our **data store** and see all the trails and the corresponding results.

In [9]:
for trial in bayes_trials:
    print(trial)
    print(20*"-")

{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.09616667032241821, 'params': {'batch_size': 50, 'learning_rate': 0.021123581053336123, 'loss_crierion': CrossEntropyLoss(), 'momentum': 0.9, 'optimizer': <class 'torch.optim.sgd.SGD'>}, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'batch_size': [0], 'learning_rate': [0]}, 'vals': {'batch_size': [2], 'learning_rate': [0.021123581053336123]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 5, 9, 18, 39, 3, 242000), 'refresh_time': datetime.datetime(2023, 5, 9, 18, 39, 15, 885000)}
--------------------
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.0989999994635582, 'params': {'batch_size': 20, 'learning_rate': 0.07618179083703541, 'loss_crierion': CrossEntropyLoss(), 'momentum': 0.9, 'optimizer': <class 'torch.optim.sgd.SGD'>}, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workd

There are two ways of fetching the best performing run (i.e. the one with the lowest loss). We can either inspect the value returned by *fmin* directly:

In [10]:
best_params

{'batch_size': 0, 'learning_rate': 0.00011394408377564594}

Or we can query the **data store** for specific values in the best performing trial.

In [11]:
bayes_trials.best_trial["result"]["loss"]

-0.9592499732971191

### Task 4

Now that we have the best hyperparameters, we need to make sure that the results are reprodicible. Inspect the values in *best_params*, reset and retrain the network. The resulting validation accuracy should be identical to the value stored in ```bayes_trials.best_trial["result"]["loss"]```.

In [12]:
net.reset()

optimizer = optim.SGD(net.parameters(), lr=best_params["learning_rate"], momentum = 0.9)    
loss_hist = train(net, 10, nn.CrossEntropyLoss(), optimizer, train_X, train_y, verbose = True)
    
acc = test(net, valid_X, valid_y)
print("Run completed. Validation accuracy: {:.10f}".format(acc))

Epochs [1/1], Samples[1000/20000], Loss: 20.0983
Epochs [1/1], Samples[2000/20000], Loss: 1.2243
Epochs [1/1], Samples[3000/20000], Loss: 0.5176
Epochs [1/1], Samples[4000/20000], Loss: 0.9346
Epochs [1/1], Samples[5000/20000], Loss: 0.1125
Epochs [1/1], Samples[6000/20000], Loss: 0.6413
Epochs [1/1], Samples[7000/20000], Loss: 0.9878
Epochs [1/1], Samples[8000/20000], Loss: 0.0091
Epochs [1/1], Samples[9000/20000], Loss: 0.0195
Epochs [1/1], Samples[10000/20000], Loss: 0.2065
Epochs [1/1], Samples[11000/20000], Loss: 0.2222
Epochs [1/1], Samples[12000/20000], Loss: 0.0680
Epochs [1/1], Samples[13000/20000], Loss: 0.0171
Epochs [1/1], Samples[14000/20000], Loss: 0.0178
Epochs [1/1], Samples[15000/20000], Loss: 0.0091
Epochs [1/1], Samples[16000/20000], Loss: 0.0015
Epochs [1/1], Samples[17000/20000], Loss: 0.0016
Epochs [1/1], Samples[18000/20000], Loss: 0.0089
Epochs [1/1], Samples[19000/20000], Loss: 0.2108
Epochs [1/1], Samples[20000/20000], Loss: 0.0695
Run completed. Validation ac

Do the accuracieis match? Compare the values of "loss" from *best_trials* and the validation that we've just computed. Remember, that the objective returns the negative accuracy so we need to correct for this. 

In [13]:
print((bayes_trials.best_trial["result"]["loss"] == -acc).item())

True


## Slightly more advanced exploration 

HyperOpt provides mechanisms for defining adaptive search spaces. This is useful when there are internal dependencies in terms of ranges (or presence) of search space hyperparameters. For example, the algorithm you are fitting can itself be a parameter of the search space. You could have a search space hyperparameter that looks like this:

```
space = hp.choice("classifier_type", [
                     {
                        "type": "svm"
                     },
                     {
                        "type": "dtree"
                     }
])
```

The issue here is that if you define your search space like this the specific hyperparameters now have a dependency on *classifier_type* (i.e. it doesn't make sense to search for the best value of *max_depth* if you are fitting an SVM model. There are two ways to handle this:

* **Option 1**: Define your search space based on the algorithm. You are essentially looking for a statement that implements the following logic:

```
 if classifier_type == "svm":
        
        search_space = {
            "C": hp.lognormal("svm_C", 0, 1),
            "kernel": hp.choice("svm_kernel", [
                {"ktype": "linear"},
                {"ktype": "RBF", "width": hp.lognormal("svm_rbf_width", 0, 1)}])
            }
        
elif classifier_type == "dtree":
        search_space = {
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "min_samples_split": hp.qlognormal("dtree_min_samples_split", 2, 1, 1)
        }
```

* **Option 2**: You nest the classifier specific hyperparameters like this:

```
search_space = hp.choice("classifier_type", [
        {
            "type": "svm",
            "C": hp.lognormal("svm_C", 0, 1),
            "kernel": hp.choice("svm_kernel", [
                {"ktype": "linear"},
                {"ktype": "RBF", "width": hp.lognormal("svm_rbf_width", 0, 1)},
                ]),
        },
        {
            "type": "dtree",
            "criterion": hp.choice("dtree_criterion", ["gini", "entropy"]),
            "min_samples_split": hp.qlognormal("dtree_min_samples_split", 2, 1, 1),
        }
    ])
```

### Task 5

In this task you'll define a search space that has the optimiser as a hyperparameter:

* HyperOpt will have to chose between two options --- optim.SGD and optim.Adagrad.
* optim.SGD should have a *momentum* parameter with possible values in [0, 0.5, 0.9]

Let's set up the search space:

In [14]:
search_space = {"learning_rate" : hp.uniform("learning_rate", 0.0001, 0.1),
                "batch_size"    : hp.choice("batch_size", [10, 20, 50, 100]),
                "optimizer"     : hp.choice("optimizer",[
                                    {"type": optim.SGD,
                                    "momentum"      : 0.9}, 
                                    {"type": optim.Adagrad}
                                  ]),
                "loss_crierion" : nn.CrossEntropyLoss()
                }

Next, we need to configure the objective.

In [15]:
def objective(params):
    
    net.reset()
    
    batch_size = params["batch_size"]
    loss_criterion = params["loss_crierion"]
    
    opt = params["optimizer"]
    
    if (opt["type"] == optim.SGD):
        optimizer = opt["type"](net.parameters(), lr=params["learning_rate"], momentum=opt["momentum"])
    else:
        optimizer = opt["type"](net.parameters(), lr=params["learning_rate"])
    
    loss_hist = train(net, batch_size, loss_criterion, optimizer, train_X, train_y, verbose = False)
    
    
    val_acc = test(net, valid_X, valid_y)
    
    print("Run completed. Validation accuracy: {:.4f}".format(val_acc))
    print(20*"-")
    
    return {"loss" : -val_acc, "params" : params, "status" : STATUS_OK} 

Finally, we call *fmin* to find the best set of hyperparameters from this extended search space. Note that we are now running 10 trials (to account for the two optimisers).

In [16]:
bayes_trials = Trials()
best_params = fmin(objective, search_space, algo=algorithm, max_evals=5, trials=bayes_trials, rstate=np.random.default_rng(1234))

Run completed. Validation accuracy: 0.0990           
--------------------                                 
Run completed. Validation accuracy: 0.8593                                      
--------------------                                                            
Run completed. Validation accuracy: 0.9196                                      
--------------------                                                           
Run completed. Validation accuracy: 0.9522                                      
--------------------                                                            
Run completed. Validation accuracy: 0.7467                                      
--------------------                                                            
100%|██████████| 5/5 [01:21<00:00, 16.32s/trial, best loss: -0.9521666765213013]


Now let's inspect the trials performed by HyperOpt.

In [17]:
for trial in bayes_trials:
    print(trial)
    print(20*"-")

{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.0989999994635582, 'params': {'batch_size': 10, 'learning_rate': 0.07953040550806391, 'loss_crierion': CrossEntropyLoss(), 'optimizer': {'momentum': 0.9, 'type': <class 'torch.optim.sgd.SGD'>}}, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'batch_size': [0], 'learning_rate': [0], 'optimizer': [0]}, 'vals': {'batch_size': [0], 'learning_rate': [0.07953040550806391], 'optimizer': [0]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 5, 9, 18, 46, 12, 13000), 'refresh_time': datetime.datetime(2023, 5, 9, 18, 46, 35, 521000)}
--------------------
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.859333336353302, 'params': {'batch_size': 100, 'learning_rate': 0.03655121639943982, 'loss_crierion': CrossEntropyLoss(), 'optimizer': {'type': <class 'torch.optim.adagrad.Adagrad'>}}, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('do

... and the best parameters

In [18]:
best_params

{'batch_size': 2, 'learning_rate': 0.006160745469131047, 'optimizer': 1}